# K-Nearest Neighbours Classification of Heart Disease

Data downloaded from the UCI Machine Learning Repository at https://archive.ics.uci.edu/ml/datasets/heart+Disease

Original data seen from Kaggle at https://www.kaggle.com/ronitf/heart-disease-uci. This data is edited and is incorrect, so is not used.

## Importing Packages

In [1]:
##Import packages

#Data Processing
import pandas as pd
import numpy as np

#Finding the data on our system
import pathlib

#KNN
from sklearn.neighbors import KNeighborsClassifier

#Scaling
from sklearn.preprocessing import StandardScaler

#Splitting data into training and test
from sklearn.model_selection import train_test_split

#Metrics to evaluate
from sklearn.metrics import accuracy_score

#Plotting library
import matplotlib.pyplot as plt

## Import Data

### Understanding the Dataset

When you collect your own data, you should know exactly what each of the variables represent, and how the variables are encoded. Unfortunately, when working with others' data, we don't start with a full understanding of the data. Not being able to understand the data makes it extremely difficult to gain insight from it. You should definitely document any data of your own, as you might have to come back to it later, or someone else might have to interpret it in the future.

As it turns out with the original dataset that was planned to be used in this example ([Kaggle Heart Disease UCI](https://www.kaggle.com/ronitf/heart-disease-uci)), it is also important to check the integrity of the dataset, especially if we plan to use the insights that we extract from the data. Resulting from investigation of the data, it seems that this source has edited data, and is misleading in various areas (for example, the indicator variable values for heart disease were swapped!). Therefore, we swap to using a correct version of the dataset, cited by multiple papers using this: [UCI Machine Learning Repository Heart Disease Dataset](https://archive.ics.uci.edu/ml/datasets/heart+Disease). This illustrates just how important it is to be able to trust our datasets if we're planning to use them.

At the UCI Machine Learning Repository, there is an accurate description of every column, of which only 14 are used (we will keep with this convention and only use these columns). The description of these columns is given as follows, to help our understanding:


- age: age in years
- sex: sex (1 = male; 0 = female)
- cp: chest pain type
1. Value 1: typical angina
2. Value 2: atypical angina
3. Value 3: non-anginal pain
4. Value 4: asymptomatic
- trestbps: resting blood pressure (in mm Hg on admission to the hospital)
- chol: serum cholestoral in mg/dl
- fbs: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
- restecg: resting electrocardiographic results
0. Value 0: normal
1. Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
2. Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
- thalach: maximum heart rate achieved
- exang: exercise induced angina (1 = yes; 0 = no)
- oldpeak = ST depression induced by exercise relative to rest
- slope: the slope of the peak exercise ST segment
1. Value 1: upsloping
2. Value 2: flat
3. Value 3: downsloping
- ca: number of major vessels (0-3) colored by flourosopy
- thal: 3 = normal; 6 = fixed defect; 7 = reversable defect
- num: diagnosis of heart disease (angiographic disease status)
0. Value 0: < 50% diameter narrowing (No disease)
1. Value 1-4: > 50% diameter narrowing (Disease)

The "num" variable is used as our target value, and as described in the description of the dataset, non-zero values here indicate disease. Therefore, we can later transform our dataset to have either a 0, or a 1 for any disease.

### Importing the Data from the CSV File

In [2]:
##Import data from another directory

#Define the columns as described
cols = [
    "age",
    "sex",
    "cp",
    "trestbps",
    "chol",
    "fbs",
    "restecg",
    "thalach",
    "exang",
    "oldpeak",
    "slope",
    "ca",
    "thal",
    "num"
]


#Find the file from a local directiory
p = str(pathlib.Path().resolve()).replace("\\Notebooks\\Classification Methods", "")
p += ("\\Data\\Raw\\Heart_Disease_UCI.csv")
p = pathlib.Path(p).resolve()

#Read into a pandas dataframe
df = pd.read_csv(p, names=cols)

#Preview the dataset
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


### Basic Data Exploration

In [3]:
#Display basic information about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    float64
 1   sex       303 non-null    float64
 2   cp        303 non-null    float64
 3   trestbps  303 non-null    float64
 4   chol      303 non-null    float64
 5   fbs       303 non-null    float64
 6   restecg   303 non-null    float64
 7   thalach   303 non-null    float64
 8   exang     303 non-null    float64
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    float64
 11  ca        303 non-null    object 
 12  thal      303 non-null    object 
 13  num       303 non-null    int64  
dtypes: float64(11), int64(1), object(2)
memory usage: 33.3+ KB


In [4]:
#Transform the target variable "num" into an indicator variable
df["num"] = [0 if i == 0 else 1 for i in df["num"]]

Notice that we have a few weird values in the dataset, such as this one. We can't use these datapoints for our analysis, and there are only a couple of them, so we will drop these rows. From our call of `df.info()`, we can see that these are in the "ca" and "thal" columns (because the data-type is object, meaning anything can occupy the position, rather than float64, specifying a decimal number), so we remove rows containing non-numeric values in these columns.

In [5]:
#Show weird value
df["thal"][266]

'?'

In [6]:
#Remove non-numeric datapoints
df = df[pd.to_numeric(df["ca"], errors='coerce').notnull()]
df = df[pd.to_numeric(df["thal"], errors='coerce').notnull()]

#Change columns to numeric
df["ca"] = pd.to_numeric(df["ca"])
df["thal"] = pd.to_numeric(df["thal"])

#Show our summary again, to check if our conversion worked
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 0 to 301
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       297 non-null    float64
 1   sex       297 non-null    float64
 2   cp        297 non-null    float64
 3   trestbps  297 non-null    float64
 4   chol      297 non-null    float64
 5   fbs       297 non-null    float64
 6   restecg   297 non-null    float64
 7   thalach   297 non-null    float64
 8   exang     297 non-null    float64
 9   oldpeak   297 non-null    float64
 10  slope     297 non-null    float64
 11  ca        297 non-null    float64
 12  thal      297 non-null    float64
 13  num       297 non-null    int64  
dtypes: float64(13), int64(1)
memory usage: 34.8 KB


## Data Transformation

### One-Hot Encoding

The data that we have has several categorical variables, which, although represented numerically, do need to be encoded. These variables have more than 2 categories that datapoints are fit into (those with 2 categories are already effectively encoded), which includes:
- cp
- restecg
- slope
- thal

We one-hot encode these using a built-in Pandas function `pd.get_dummies()`. We don't yet add these variables back into the DataFrame because of the scaling step that we will perform later, that is not appropriate for categorical variables such as these. Later, we will reconstruct a full DataFrame, such that we can train our KNN model.

In [7]:
#One-hot encoding
cp_dummies = pd.get_dummies(df["cp"], prefix="cp")
restecg_dummies = pd.get_dummies(df["restecg"], prefix="restecg")
slope_dummies = pd.get_dummies(df["slope"], prefix="slope")
thal_dummies = pd.get_dummies(df["thal"], prefix="thal")

#Drop the original variables from the DataFrame
df = df.drop(["cp", "restecg", "slope", "thal"], axis=1)
df

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,ca,num
0,63.0,1.0,145.0,233.0,1.0,150.0,0.0,2.3,0.0,0
1,67.0,1.0,160.0,286.0,0.0,108.0,1.0,1.5,3.0,1
2,67.0,1.0,120.0,229.0,0.0,129.0,1.0,2.6,2.0,1
3,37.0,1.0,130.0,250.0,0.0,187.0,0.0,3.5,0.0,0
4,41.0,0.0,130.0,204.0,0.0,172.0,0.0,1.4,0.0,0
...,...,...,...,...,...,...,...,...,...,...
297,57.0,0.0,140.0,241.0,0.0,123.0,1.0,0.2,0.0,1
298,45.0,1.0,110.0,264.0,0.0,132.0,0.0,1.2,0.0,1
299,68.0,1.0,144.0,193.0,1.0,141.0,0.0,3.4,2.0,1
300,57.0,1.0,130.0,131.0,0.0,115.0,1.0,1.2,1.0,1
